In [21]:
#import required libraries

import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
from pathlib import Path
from beasy.beasy import Bea
%matplotlib inline

In [22]:
#load the API keys stored in .env file
load_dotenv()

True

In [3]:
# https://apps.bea.gov/api/signup/index.cfm to signup for API key for Bureau of Economic Analysis

# https://apps.bea.gov/api/_pdf/bea_web_service_api_user_guide.pdf for user guide

In [23]:
#get BEA API key from .env file
bea_api_key=os.getenv("BEA_API_KEY")

#Defining the beasy client to read the BEA Data using the Beasy API reader
client = Bea(bea_api_key)

In [24]:
# set up query URL
bea_url = "http://apps.bea.gov/api/data?UserID=" + bea_api_key + "&method=GETDATASETLIST&ResultFormat=JSON" 

In [25]:
#send request to BEA
bea_response = requests.get(bea_url).json()

In [26]:
#review response
bea_response

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'USERID',
     'ParameterValue': 'ED49F4D3-70D1-4E41-94A5-B625BCB7A72B'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATASETLIST'}]},
  'Results': {'Dataset': [{'DatasetName': 'NIPA',
     'DatasetDescription': 'Standard NIPA tables'},
    {'DatasetName': 'NIUnderlyingDetail',
     'DatasetDescription': 'Standard NI underlying detail tables'},
    {'DatasetName': 'MNE', 'DatasetDescription': 'Multinational Enterprises'},
    {'DatasetName': 'FixedAssets',
     'DatasetDescription': 'Standard Fixed Assets tables'},
    {'DatasetName': 'ITA',
     'DatasetDescription': 'International Transactions Accounts'},
    {'DatasetName': 'IIP',
     'DatasetDescription': 'International Investment Position'},
    {'DatasetName': 'InputOutput', 'DatasetDescription': 'Input-Output Data'},
    {'DatasetName': 'IntlServTrade',
     'DatasetDescription': 'International S

In [27]:
#Beasy client to get the BEA Data set list
client.getDatasetList()

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'USERID',
     'ParameterValue': 'ED49F4D3-70D1-4E41-94A5-B625BCB7A72B'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATASETLIST'}]},
  'Results': {'Dataset': [{'DatasetName': 'NIPA',
     'DatasetDescription': 'Standard NIPA tables'},
    {'DatasetName': 'NIUnderlyingDetail',
     'DatasetDescription': 'Standard NI underlying detail tables'},
    {'DatasetName': 'MNE', 'DatasetDescription': 'Multinational Enterprises'},
    {'DatasetName': 'FixedAssets',
     'DatasetDescription': 'Standard Fixed Assets tables'},
    {'DatasetName': 'ITA',
     'DatasetDescription': 'International Transactions Accounts'},
    {'DatasetName': 'IIP',
     'DatasetDescription': 'International Investment Position'},
    {'DatasetName': 'InputOutput', 'DatasetDescription': 'Input-Output Data'},
    {'DatasetName': 'IntlServTrade',
     'DatasetDescription': 'International Services Trade'},
    {'DatasetName': 'GDPbyIndustry', 'DatasetDes

In [ ]:
# the following code was used to test the response for regional data

In [28]:
 bea_regional_url = "http://apps.bea.gov/api/data?UserID=" + bea_api_key + "&method=getparameterlist&datasetname=Regional&ResultFormat=JSON" 

In [29]:
 bea_regional_response = requests.get(bea_regional_url).json()

In [30]:
 bea_regional_response

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'RESULTFORMAT',
     'ParameterValue': 'JSON'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'USERID',
     'ParameterValue': 'ED49F4D3-70D1-4E41-94A5-B625BCB7A72B'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETPARAMETERLIST'}]},
  'Results': {'Parameter': [{'ParameterName': 'GeoFips',
     'ParameterDataType': 'string',
     'ParameterDescription': 'Comma-delimited list of 5-character geographic codes; COUNTY for all counties, STATE for all states, MSA for all MSAs, MIC for all Micropolitan Areas, PORT for all state metro/nonmetro portions, DIV for all Metropolitan Divisions, CSA for all Combined Statistical Areas, state post office abbreviation for all counties in one state (e.g. NY)',
     'ParameterIsRequiredFlag': '1',
     'MultipleAcceptedFlag': '1'},
    {'ParameterName': 'LineCode',
     'ParameterDataType': 'integer',
     'ParameterDescription': 'Line code for a 

In [85]:
# the following code was used to get FIPS codes. May not be necessary but leaving it for now in case it is needed later.

In [11]:
# https://www.census.gov/geographies/reference-files/2016/demo/popest/2016-fips.html
    
# xml file location for GeoFIPS state and locality codes

In [12]:
# example api call from bea api instructions pdf, to be edited to get data for gdp per capita for all states

# https://apps.bea.gov/api/data/?&UserID=Your-36CharacterKey&method=GetData&datasetname=Regional&TableName=CAINC1&LineCode=3&GeoFIPS=DE&Year=2014&ResultFormat=JSON
                
# the following is an attempt to get linecodes for a call.    
    
# https://apps.bea.gov/api/data/?&UserID=Your-36CharacterKey&method=GetData&datasetname=Regional&TableName=SQINC1&TargetParameter=Linecode&ResultFormat=JSON    

In [13]:
# this one does not work, needs more parameters

# bea_states_personal_income_linecode_query_url = "http://apps.bea.gov/api/data?UserID=" + bea_api_key + "&method=GetData&datasetname=Regional&TableName=SAINC1&GeoFIPS=STATE&LineCode=1&Year=ALL&ResultFormat=JSON"

In [14]:
# the following query works

# querytest = "http://apps.bea.gov/api/data?&UserID=" + bea_api_key + "&method=GetData&datasetname=Regional&TableName=SAINC1&GeoFIPS=STATE&LineCode=3&Year=2013&ResultFormat=JSON"

In [15]:
# requests.get(querytest).json()

In [16]:
# bea_personal_income_linecode_query_response = requests.get(bea_states_personal_income_linecode_query_url).json()

In [17]:
# bea__personal_income_linecode_query_response

In [18]:
# the following code was written to extract FIPS codes. It may not be needed.

In [19]:
# geofips_table = pd.read_excel(Path("./Resources/state-geocodes-v2016.xls"))

In [20]:
#geofips_table

In [21]:
# to view full geofips table in jupyter lab

# geofips_table.to_csv("geofips.csv")

In [22]:
# Codes (from the BEA API instructions pdf):

# SQINC1 Personal Income Summary: Personal Income, Population, Per Capita Personal Income by States

# SQGDP9 Real GDP by state 

# SQGDP2 Gross domestic product (GDP) by state 2

In [31]:
# this query will get the data for all states for the last five years for personal income per capita.

bea_states_personal_income_linecode_query_url = "http://apps.bea.gov/api/data?UserID=" + bea_api_key + "&method=GetData&datasetname=Regional&TableName=SQINC1&GeoFIPS=STATE&LineCode=3&Year=LAST5&ResultFormat=JSON"

In [32]:
#query for last five years personal income by state.
personal_income_by_state_5year = requests.get(bea_states_personal_income_linecode_query_url).json()

In [33]:
#review the results of the query
personal_income_by_state_5year

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'YEAR',
     'ParameterValue': 'LAST5'},
    {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'},
    {'ParameterName': 'TABLENAME', 'ParameterValue': 'SQINC1'},
    {'ParameterName': 'GEOFIPS', 'ParameterValue': 'STATE'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'LINECODE', 'ParameterValue': '3'},
    {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'},
    {'ParameterName': 'USERID',
     'ParameterValue': 'ED49F4D3-70D1-4E41-94A5-B625BCB7A72B'}]},
  'Results': {'Statistic': 'Per capita personal income',
   'UnitOfMeasure': 'Dollars',
   'PublicTable': 'SQINC1 Personal Income Summary: Personal Income, Population, Per Capita Personal Income',
   'UTCProductionTime': '2021-11-11T04:15:39.303',
   'NoteRef': '2',
   'Dimensions': [{'Name': 'Code', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoFips', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoNam

In [19]:
# Using the Beasy client to read the Personal Income data from the BEA API

client.Regional.getData(TableName='SQINC1', LineCode='3', GeoFIPS='STATE', Year='LAST5')

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'METHOD',
     'ParameterValue': 'GETDATA'},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'},
    {'ParameterName': 'TABLENAME', 'ParameterValue': 'SQINC1'},
    {'ParameterName': 'GEOFIPS', 'ParameterValue': 'STATE'},
    {'ParameterName': 'LINECODE', 'ParameterValue': '3'},
    {'ParameterName': 'USERID',
     'ParameterValue': 'ED49F4D3-70D1-4E41-94A5-B625BCB7A72B'},
    {'ParameterName': 'YEAR', 'ParameterValue': 'LAST5'}]},
  'Results': {'Statistic': 'Per capita personal income',
   'UnitOfMeasure': 'Dollars',
   'PublicTable': 'SQINC1 Personal Income Summary: Personal Income, Population, Per Capita Personal Income',
   'UTCProductionTime': '2021-11-11T04:11:57.730',
   'NoteRef': '2',
   'Dimensions': [{'Name': 'Code', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoFips', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'GeoName', 'DataType': 'string', 'IsValue': '0'},
    {'Name': 'TimePeri

In [178]:
# extract main data from json
personal_income_by_state_5year_unsorted = pd.DataFrame(personal_income_by_state_5year["BEAAPI"]["Results"]["Data"])
personal_income_by_state_5year_unsorted

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue,NoteRef
0,SQINC1-3,00000,United States,2018Q4,Dollars,0,"54,874",NaN
1,SQINC1-3,00000,United States,2018Q3,Dollars,0,"54,409",NaN
2,SQINC1-3,00000,United States,2018Q2,Dollars,0,"53,807",NaN
3,SQINC1-3,00000,United States,2018Q1,Dollars,0,"53,287",NaN
4,SQINC1-3,00000,United States,2017Q1,Dollars,0,"51,018",NaN
...,...,...,...,...,...,...,...,...
1075,SQINC1-3,98000,Far West,2017Q4,Dollars,0,"57,919",NaN
1076,SQINC1-3,98000,Far West,2018Q4,Dollars,0,"60,639",NaN
1077,SQINC1-3,98000,Far West,2018Q3,Dollars,0,"60,178",NaN
1078,SQINC1-3,98000,Far West,2018Q2,Dollars,0,"59,483",NaN


In [179]:
# clip out the unnecessary columns
personal_income_by_state_5year_unsorted=personal_income_by_state_5year_unsorted[["GeoName", "TimePeriod", "DataValue"]]
personal_income_by_state_5year_unsorted


,GeoName,TimePeriod,DataValue
0,United States,2018Q4,"54,874"
1,United States,2018Q3,"54,409"
2,United States,2018Q2,"53,807"
3,United States,2018Q1,"53,287"
4,United States,2017Q1,"51,018"
...,...,...,...
1075,Far West,2017Q4,"57,919"
1076,Far West,2018Q4,"60,639"
1077,Far West,2018Q3,"60,178"
1078,Far West,2018Q2,"59,483"


In [36]:
# data cleaning: change time periods to be .25 for Q1, etc
# change the comma format numbers to float

In [180]:
for index, row in personal_income_by_state_5year_unsorted.items():
    print(index)
    print("\n")
    print(row)


# for index, row in personal_income_by_state_5year_unsorted:
#     print(index)
    #     line["DataValue"] = line["DataValue"].replace(',', '')

# personal_income_by_state_5year_unsorted["DataValue"] = float(personal_income_by_state_5year_unsorted["DataValue"].replace(",", ""))

GeoName


0       United States
1       United States
2       United States
3       United States
4       United States
            ...      
1075         Far West
1076         Far West
1077         Far West
1078         Far West
1079         Far West
Name: GeoName, Length: 1080, dtype: object
TimePeriod


0       2018Q4
1       2018Q3
2       2018Q2
3       2018Q1
4       2017Q1
         ...  
1075    2017Q4
1076    2018Q4
1077    2018Q3
1078    2018Q2
1079    2018Q1
Name: TimePeriod, Length: 1080, dtype: object
DataValue


0       54,874
1       54,409
2       53,807
3       53,287
4       51,018
         ...  
1075    57,919
1076    60,639
1077    60,178
1078    59,483
1079    58,819
Name: DataValue, Length: 1080, dtype: object


In [173]:
# for row in personal_income_by_state_5year_unsorted.rows:
# #     personal_income_by_state_5year_df[item["TimePeriod"]][item["GeoName"]]=item["DataValue"]
#     print(row)
    
#for index, row in personal_income_by_state_5year_unsorted.iterrows():
#    personal_income_by_state_5year_table[row["TimePeriod"],row["GeoName"]]=row["DataValue"]
                       
#                                       print(row['c1'], row['c2'])
#print(personal_income_by_state_5year_table)

In [181]:
# Used str.replace() to remove the ',' from the Income datavalue and convert to float from string

personal_income_by_state_5year_unsorted['DataValue'] = personal_income_by_state_5year_unsorted['DataValue'].str.replace(',', '').astype('float')
personal_income_by_state_5year_unsorted['DataValue']

C:\Users\abhis\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


0       54874.0
1       54409.0
2       53807.0
3       53287.0
4       51018.0
         ...   
1075    57919.0
1076    60639.0
1077    60178.0
1078    59483.0
1079    58819.0
Name: DataValue, Length: 1080, dtype: float64

In [182]:
personal_income_by_state_5year["BEAAPI"]["Results"]["Data"]

[{'Code': 'SQINC1-3',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2018Q4',
  'CL_UNIT': 'Dollars',
  'UNIT_MULT': '0',
  'DataValue': '54,874'},
 {'Code': 'SQINC1-3',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2018Q3',
  'CL_UNIT': 'Dollars',
  'UNIT_MULT': '0',
  'DataValue': '54,409'},
 {'Code': 'SQINC1-3',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2018Q2',
  'CL_UNIT': 'Dollars',
  'UNIT_MULT': '0',
  'DataValue': '53,807'},
 {'Code': 'SQINC1-3',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2018Q1',
  'CL_UNIT': 'Dollars',
  'UNIT_MULT': '0',
  'DataValue': '53,287'},
 {'Code': 'SQINC1-3',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2017Q1',
  'CL_UNIT': 'Dollars',
  'UNIT_MULT': '0',
  'DataValue': '51,018'},
 {'Code': 'SQINC1-3',
  'GeoFips': '00000',
  'GeoName': 'United States',
  'TimePeriod': '2017Q2',
  'CL_UNIT': 'Dollars',
  'UNIT_MULT': '0',
  'Dat

In [183]:
# Filter the dataframe to get just the states

list_of_states = [ 'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'Florida', 'Georgia',                            'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',                        'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 
                   'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island',
                   'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia',                            'Wisconsin', 'Wyoming']

personal_income_state_filter = personal_income_by_state_5year_unsorted['GeoName'].isin(list_of_states)
personal_income_filter_by_state = personal_income_by_state_5year_unsorted[personal_income_state_filter]
print(personal_income_filter_by_state)

     GeoName TimePeriod  DataValue
18   Alabama     2020Q1    45031.0
19   Alabama     2020Q2    48797.0
20   Alabama     2020Q3    46288.0
21   Alabama     2020Q4    45736.0
22   Alabama     2019Q1    43530.0
..       ...        ...        ...
931  Wyoming     2021Q2    63589.0
932  Wyoming     2017Q1    55381.0
933  Wyoming     2017Q2    56299.0
934  Wyoming     2017Q3    57082.0
935  Wyoming     2017Q4    58031.0

[864 rows x 3 columns]


In [165]:
# Calculating a float equivalent of the quarters by iterating into the dataframe
personal_income_by_state_time_wt = personal_income_filter_by_state

for index, row in personal_income_by_state_time_wt.iterrows():
    if 'Q1' in row['TimePeriod']:
        q1 = 0.25
        personal_income_by_state_time_wt.at[index, 'TimePeriod_wt'] = q1
    elif 'Q2' in row['TimePeriod']:
        q2 = 0.5
        personal_income_by_state_time_wt.at[index, 'TimePeriod_wt'] = q2
    elif 'Q3' in row['TimePeriod']:
        q3 = 0.75
        personal_income_by_state_time_wt.at[index, 'TimePeriod_wt'] = q3
    elif 'Q4' in row['TimePeriod']:
        q4 = 1.0
        personal_income_by_state_time_wt.at[index, 'TimePeriod_wt'] = q4
    
personal_income_by_state_time_wt
        

C:\Users\abhis\Anaconda3\envs\dev\lib\site-packages\pandas\core\indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\abhis\Anaconda3\envs\dev\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,GeoName,TimePeriod,DataValue,TimePeriod_wt
18,Alabama,2020Q1,45031.0,0.25
19,Alabama,2020Q2,48797.0,0.50
20,Alabama,2020Q3,46288.0,0.75
21,Alabama,2020Q4,45736.0,1.00
22,Alabama,2019Q1,43530.0,0.25
...,...,...,...,...
931,Wyoming,2021Q2,63589.0,0.50
932,Wyoming,2017Q1,55381.0,0.25
933,Wyoming,2017Q2,56299.0,0.50
934,Wyoming,2017Q3,57082.0,0.75


In [184]:
# Setting the time period to be the index and converting the datatype from string to dateTime

personal_income_filter_by_state = personal_income_filter_by_state.set_index('TimePeriod')
personal_income_filter_by_state.index = pd.to_datetime(personal_income_filter_by_state.index)
personal_income_filter_by_state

,GeoName,DataValue
TimePeriod,,
2020-01-01,Alabama,45031.0
2020-04-01,Alabama,48797.0
2020-07-01,Alabama,46288.0
2020-10-01,Alabama,45736.0
2019-01-01,Alabama,43530.0
...,...,...
2021-04-01,Wyoming,63589.0
2017-01-01,Wyoming,55381.0
2017-04-01,Wyoming,56299.0


In [185]:
# Group the data by state and calculate the average personal income for every year from 2017 to current year

personal_income_filter_annual = personal_income_filter_by_state.groupby('GeoName').resample('A').mean()
personal_income_filter_annual

DataValue
GeoName TimePeriod           
Alabama 2017-12-31   40855.25
        2018-12-31   42319.25
        2019-12-31   43987.75
        2020-12-31   46463.00
        2021-12-31   50383.00
...                       ...
Wyoming 2017-12-31   56698.25
        2018-12-31   59877.25
        2019-12-31   61036.25
        2020-12-31   61838.00
        2021-12-31   65808.50

[240 rows x 1 columns]